# **scrython_bulk_data_etl**

---

<br><br><br><br>

## **Objectives**

---


<br><br><br><br>

## **TO DOs**

---

- [x] 2022-10-03 23:37:15: setup proc `[scrython_bulk_data_etl]`: 2022-10-01
    
- [x] 2022-10-03 23:37:21: setup gitignore to ignore the json data dump and our saved parquet file

- TODO [ ] finish final docs on this proc: 2022-10-03

<br><br><br><br><br><br>

## **Imports / Setup Environment**
---

In [7]:
import pandas as pd
from fp_data_toolbox import eda, notifier

### **Variable Setup**

In [8]:
from datetime import datetime
# datetime object containing current date and time
now = datetime.now()
ts_string = now.strftime("%Y-%m-%d %H-%M-%S")
dt_string = now.strftime("%Y-%m-%d")

###=============================================
### variables for feather file pathing

resources_dir = 'C:\\git\\mtg-proj\\mtg-etl\\resources\\scryfall_api_data'
default_cards_feather_name='default_cards-'+dt_string+'.feather'
oracle_cards_feather_name='oracle_cards-'+dt_string+'.feather'

default_cards_feather_path = resources_dir+'\\'+default_cards_feather_name
oracle_cards_feather_path = resources_dir+'\\'+oracle_cards_feather_name

###=============================================
### variables for JSON file pathing

default_cards_json_name='default_cards-'+dt_string+'.json'
oracle_cards_json_name='oracle_cards-'+dt_string+'.json'

default_cards_json_wildcard='default_cards*'
oracle_cards_json_wildcard='oracle_cards*'

default_cards_json_path = resources_dir+'\\'+default_cards_json_name
oracle_cards_json_path = resources_dir+'\\'+oracle_cards_json_name

default_cards_json_path_wc = resources_dir+'\\'+default_cards_json_wildcard
oracle_cards_json_path_wc = resources_dir+'\\'+oracle_cards_json_wildcard


<br><br><br><br>

## **Flow Control**

---

- [ ] check whether scryfall mass data file exists; if no, start create script


In [9]:
bool_default_cards_json_exists = False
bool_oracle_cards_json_exists = False
import os
for filename in os.scandir(resources_dir):
    if filename.is_file():
        filename_str=filename.path.rsplit('\\', 1)[1] or filename.path 
        filename_str=filename_str.rsplit('.', 1)[0] or filename_str # find the filename 
        print(filename_str)
        if filename_str.startswith('default_cards') and filename.path.endswith('.json'):
            file_dt_str=filename_str.rsplit('-', 3)[1]+'-'+filename_str.rsplit('-', 2)[1]+'-'+filename_str.rsplit('-', 1)[1]
            if file_dt_str == dt_string:
            # TODO [ ] add logic for matching on date as well
                bool_default_cards_json_exists = True
                continue
        if filename_str.startswith('oracle_cards') and filename.path.endswith('.json'):
            file_dt_str=filename_str.rsplit('-', 3)[1]+'-'+filename_str.rsplit('-', 2)[1]+'-'+filename_str.rsplit('-', 1)[1]
            if file_dt_str == dt_string:
            # TODO [ ] add logic for matching on date as well
                bool_oracle_cards_json_exists = True
                continue

default_cards-2022-10-03
default_cards-2022-10-03
oracle_cards-2022-10-03
oracle_cards-2022-10-03



<br><br><br><br>

## **Scrython Data Ingestion**

---

- [x] download api data dump as json
- [x] convert to df

In [10]:
# function for downloading json files from url
def download_json_from_https(scryfall_bulk_data_name,path_out):
    import requests
    import os
    import nest_asyncio
    
    nest_asyncio.apply()
    i = 0
    import scrython

    # setup method for pulling metadata from scrython
    data_info = scrython.bulk_data.BulkData()

    # pull scryfall metadata into string
    scryfall_metadata_str = data_info.data()

    # converting str to json and writing to .json file
    import json
    json_object = json.dumps(scryfall_metadata_str, indent=4)
    with open("scryfall_metadata.json", "w") as outfile:
        outfile.write(json_object)

    # reading .json file as df
    df_scryfall_metadata = pd.read_json("scryfall_metadata.json").set_index('name')    

    download_uri = df_scryfall_metadata.loc[scryfall_bulk_data_name, 'download_uri']

    print(scryfall_bulk_data_name)
    #download url
    r = requests.get(download_uri, allow_redirects=True)
    # writing to json
    open(path_out, 'wb').write(r.content)
    # reading json as df
    df = pd.read_json(path_out)
    # deleting temp json file
    # os.remove(temp_filename)
    return df

In [11]:
## downloading bulk data from scryfall (if not already cached)
default_cards_bulk_data_name = 'Default Cards'
oracle_cards_bulk_data_name = 'Oracle Cards'

import glob

###=============================================
if bool_default_cards_json_exists == False:
    
    # get a recursive list of file paths that matches pattern including sub directories
    fileList = glob.glob('C:/git/mtg-proj/mtg-etl/resources/scryfall_api_data/default_cards*.json', recursive=True)
    # Iterate over the list of filepaths & remove each file.
    for filePath in fileList:
        try:
            os.remove(filePath)
        except OSError:
            print("Error while deleting file")

    # os.remove(default_cards_json_path_wc)
    print('starting default_cards bulk_data pull; might take a few seconds')
    df_default_cards = download_json_from_https(default_cards_bulk_data_name,default_cards_json_path)
    print('done pulling default_cards bulk_data ; no errors')
else:
    print('default_cards already cached; won"t pull again')
    df_default_cards = pd.read_json(default_cards_json_path)
###=============================================

if bool_oracle_cards_json_exists == False:

    # get a recursive list of file paths that matches pattern including sub directories
    fileList = glob.glob('C:/git/mtg-proj/mtg-etl/resources/scryfall_api_data/oracle_cards*.json', recursive=True)
    # Iterate over the list of filepaths & remove each file.
    for filePath in fileList:
        try:
            os.remove(filePath)
        except OSError:
            print("Error while deleting file")

    # os.remove(oracle_cards_json_path_wc)
    print('starting oracle_cards bulk_data pull; might take a few seconds')
    df_oracle_cards = download_json_from_https(oracle_cards_bulk_data_name,oracle_cards_json_path)
    print('done pulling oracle_cards bulk_data ; no errors')
else:
    print('oracle_cards bulk_data already cached; won"t pull again')
    df_oracle_cards = pd.read_json(oracle_cards_json_path)
###=============================================


default_cards already cached; won"t pull again
oracle_cards bulk_data already cached; won"t pull again


In [12]:
#stop

<br><br><br><br>

## **Cleaning / Transform**

---

- [ ] set proper indexes


<br>

- [ ] split `prices` field into multiple price fields

- [ ] optimize df for parquet storage
    - [ ] cast best datatypes

- [ ] add any features we need to create ourselvess

<br><br><br><br><br>


---

- TODO [ ] summarize all cleaning operations into one scryfall_bulk_data_clean function: 2022-10-02

In [13]:
def scryfall_bulk_data_clean(df_input):
    df=df_input
    df=df.rename(columns={
        # DONE [x] clean up names (renaming): 2022-10-02
        "uri": "api_uri",
        "lang": "language",
        # add more column name changes here
        "released_at": "released_date"
    }, errors="raise")
    
    
    ###=============================================
    ### DONE [x]  fields to unpack;noted_on:2022-10-02
        ### [x]  'prices'
    ## unpack fields into new data frame
    df_prices=pd.DataFrame()
    df_prices=pd.DataFrame(df['prices'].to_list(), columns=[
        'usd', 
        'usd_foil', 
        'usd_etched', 
        # 'eur', 
        # 'eur_foil', 
        'tix'
        ])
    df_prices['id']=df['id']
    ### merge df_prices back into main df
    df=df.merge(df_prices, on='id', how='left')
    # del df_prices
    
    ###=============================================
    ### [x]  unpack  'related_uris'
    df_uris=pd.DataFrame()
    df_uris=pd.DataFrame(df['related_uris'].to_list(), columns=[
        'gatherer', 
        # 'tcgplayer_infinite_articles', 
        # 'tcgplayer_infinite_decks', 
        'edhrec'
        ])
    df_uris['id']=df['id']
    ### rename columns
    df_uris=df_uris.rename(columns={
        "gatherer": "gatherer_uri",
        # "tcgplayer_infinite_articles": "tcgplayer_infinite_articles_uri", 
        # "tcgplayer_infinite_decks": "tcgplayer_infinite_decks_uri", 
        "edhrec": "edhrec_uri"
    }, errors="raise")
    ### merge df_uris back into main df
    df=df.merge(df_uris, on='id', how='left')
    # del df_uris
    
    ###=============================================
    ### SOMEDAY unpack 'legalities'
    
    ### SOMEDAY unpack 'games'
    
    
    
    ###=============================================
    # - [x] eliminate completely unnecessary fields
        # - `related_uri`
    df=df.drop(columns=[
        # add more fields to drop here
        'object',
        'highres_image',
        'color_indicator',
        'hand_modifier',
        'life_modifier',
        'loyalty',
        'image_status',
        'image_uris',
        'prints_search_uri',
        'artist_ids',
        'illustration_id',
        'tcgplayer_etched_id',
        'preview',
        'all_parts',
        'promo_types',
        'content_warning',
        'story_spotlight',
        'textless',
        'attraction_lights',
    
    ### drop unpacked columns from main df
        'related_uris',
        'legalities',
        'games',
        'prices'
    ])

    ###=============================================
    ### [ ]  cast the correct data types here
        ### 'df.astype'
        ### [x] prices as downcast float
        ### [ ] categorical fields as category
        ### [x] boolean fields as boolean
    
    ## float fields
    df['usd'] = pd.to_numeric(df['usd'], downcast='float')
    df['usd_foil'] = pd.to_numeric(df['usd_foil'], downcast='float')
    df['usd_etched'] = pd.to_numeric(df['usd_etched'], downcast='float')
    df['tix'] = pd.to_numeric(df['tix'], downcast='float')
    df['cmc'] = pd.to_numeric(df['cmc'], downcast='float')
    # df['power'] = pd.to_numeric(df['power'], downcast='float')        # not actually a float field (strings exist)
    # df['tougness'] = pd.to_numeric(df['tougness'], downcast='float')  # not actually a float field (strings exist)
    
    ## integer fields
    df['penny_rank'] = pd.to_numeric(df['penny_rank'], downcast='integer')
    df['edhrec_rank'] = pd.to_numeric(df['edhrec_rank'], downcast='integer')
    df['cardmarket_id'] = pd.to_numeric(df['cardmarket_id'], downcast='integer')
    df['tcgplayer_id'] = pd.to_numeric(df['tcgplayer_id'], downcast='integer')
    df['arena_id'] = pd.to_numeric(df['arena_id'], downcast='integer')
    
    # ## category fields
    df['set_type'] =            df['set_type'].astype('category')
    df['language'] =            df['language'].astype('category')
    df['security_stamp'] =      df['security_stamp'].astype('category')
    df['rarity'] =              df['rarity'].astype('category')
    # df['frame_effects'] =       df['frame_effects'].astype('category')    # actually a list data type, not a string
    df['layout'] =              df['layout'].astype('category')
    # df['finishes'] =            df['finishes'].astype('category')         # actually a list data type, not a string
    df['border_color'] =        df['border_color'].astype('category')
    df['frame'] =               df['frame'].astype('category')
    df['watermark'] =           df['watermark'].astype('category')
    
    # ## boolean fields    
    df['reserved'] =            df['reserved'].astype('boolean')
    df['foil'] =                df['foil'].astype('boolean')
    df['nonfoil'] =             df['nonfoil'].astype('boolean')
    df['oversized'] =           df['oversized'].astype('boolean')
    df['promo'] =               df['promo'].astype('boolean')
    df['reprint'] =             df['reprint'].astype('boolean')
    df['variation'] =           df['variation'].astype('boolean')
    df['digital'] =             df['digital'].astype('boolean')
    df['full_art'] =            df['full_art'].astype('boolean')
    df['booster'] =             df['booster'].astype('boolean')
    
    # ## object fields
    df['colors'] =              df['colors'].astype('object')
    df['color_identity'] =      df['color_identity'].astype('object')
    df['keywords'] =            df['keywords'].astype('object')
    df['produced_mana'] =       df['produced_mana'].astype('object')
    df['card_faces'] =          df['card_faces'].astype('object')
    df['frame_effects'] =       df['frame_effects'].astype('object')
    df['finishes'] =            df['finishes'].astype('object')
    df['multiverse_ids'] =      df['multiverse_ids'].astype('object')
    
    ###=============================================
    ### transform 'set' field to uppercase    
    df['set']=df['set'].str.upper()

    ### [x] reorder columns 
    df=df[[
        'id',
        'name',
        
        ###=============================================
        ### price data
        'usd',
        'usd_foil',
        'usd_etched',
        'tix',
        
        ###=============================================
        ### on-card / strategic data
        'cmc',
        'mana_cost',
        'colors',
        'color_identity',
        'power',
        'toughness',
        'type_line',
        'keywords',
        'produced_mana',
        'oracle_text',
        
        ###=============================================
        ### rank data
        'penny_rank',
        'edhrec_rank',
        
        ###=============================================
        ### collector fields
        'collector_number',
        'set',
        'set_name',
        'set_type',
        'released_date',
        'language',
        'security_stamp',
        'rarity',
        'card_faces',
        'frame_effects',
        'layout',
        'watermark',
        'finishes',
        'artist',
        'border_color',
        'frame',
        'flavor_text',        
        
        ### boolean collector fields
        'reserved',
        'foil',
        'nonfoil',
        'oversized',
        'promo',
        'reprint',
        'variation',
        'digital',
        'full_art',
        'booster',
        
        ###=============================================
        ### ids
        'oracle_id',
        'set_id',
        'mtgo_id',
        'mtgo_foil_id',
        'cardmarket_id',
        'tcgplayer_id',
        
        'arena_id',
        'card_back_id',
        'multiverse_ids',
        
        ###=============================================
        ### uris
        'api_uri',
        'set_uri',
        'set_search_uri',
        'scryfall_uri',
        'scryfall_set_uri',
        'rulings_uri',        
        'gatherer_uri',
        'edhrec_uri'
    ]]
    return df

In [14]:
df_default_cards=scryfall_bulk_data_clean(df_default_cards)
df_oracle_cards=scryfall_bulk_data_clean(df_oracle_cards)

In [15]:
# - [x] clean up `id` dimensionality for `df_oracle_cards`
#     - needs to be completely distinct on `id`
# - [x] ways of eliminating `id` duplicates
#         - group by `id` for min `price_usd` into new df

df_default_cards_stg = pd.DataFrame()
df_default_cards_stg['id']=df_default_cards['id']
df_default_cards_stg['usd']=df_default_cards['usd']

df_default_cards_id=df_default_cards_stg.groupby(by='id').min()

df_default_cards = pd.merge(df_default_cards, df_default_cards_id, on='id', how='inner')

df_default_cards=df_default_cards.rename(columns={
        "usd_x": "usd"
    }, errors="raise")

In [16]:
# - [x] clean up `name` dimensionality for `df_default_cards`
#   - needs to be completely distinct on `name`
# - [x] eliminating `name` duplicates
#     - group by `name` for max `id` into new df

df_oracle_cards_stg = pd.DataFrame()
df_oracle_cards_stg['name']=df_oracle_cards['name']
df_oracle_cards_stg['id']=df_oracle_cards['id']

df_oracle_cards_id=df_oracle_cards_stg.groupby(by='name').max()

df_oracle_cards = pd.merge(df_oracle_cards, df_oracle_cards_id, on='id', how='inner')

In [17]:
### [x] sort and reindex
df_default_cards=df_default_cards.sort_values(by=['name','id'])
df_default_cards=df_default_cards.reset_index(drop=True)

df_oracle_cards=df_oracle_cards.sort_values(by=['name','id'])
df_oracle_cards=df_oracle_cards.reset_index(drop=True)


In [18]:
### set index fields
# df_default_cards = df_default_cards.set_index('name')
# df_default_cards['name'] = df_default_cards.index

# df_oracle_cards = df_oracle_cards.set_index('id')
# df_oracle_cards['id'] = df_oracle_cards.index

In [19]:
# # df=df_oracle_cards
# df=df_default_cards
# eda.copi_df(df)
# df.info()

In [20]:
#stop

<br><br><br><br>

## **Outputs**

---

- TODO [ ] save as feather file in local storage
    - [ ] decide on location of feather output
        - can this be integrated within python modules that can be imported by other procs?
    - [ ] create var for feather output

In [21]:
# import feather

### TODO [ ] logic for skipping scrython_bulk_data_etl processing ;noted_on:2022-10-03
    ### when feather file all ready present and date ts matches
df_default_cards.to_feather(default_cards_feather_path)
df_oracle_cards.to_feather(oracle_cards_feather_path)

# df_oracle_cards.to_feather()

### **Save to Archive**

In [22]:
# not sure if we'll need this
# fill in

In [23]:
#stop

<br><br><br><br>

## **Unit Test**

---

Unit tests here

In [24]:
df_default_cards_test = pd.read_feather(default_cards_feather_path)
df_oracle_cards_test = pd.read_feather(oracle_cards_feather_path)
# fill in

In [26]:
df = df_default_cards
# df = df_oracle_cards
eda.copi_df(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73388 entries, 0 to 73387
Data columns (total 63 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id                73388 non-null  object        
 1   name              73388 non-null  object        
 2   usd               55034 non-null  float32       
 3   usd_foil          36264 non-null  float32       
 4   usd_etched        729 non-null    float32       
 5   tix               35738 non-null  float32       
 6   cmc               73383 non-null  float32       
 7   mana_cost         71423 non-null  object        
 8   colors            71423 non-null  object        
 9   color_identity    73388 non-null  object        
 10  power             34333 non-null  object        
 11  toughness         34333 non-null  object        
 12  type_line         73383 non-null  object        
 13  keywords          73388 non-null  object        
 14  produced_mana     1069

In [ ]:
#stop

<br><br><br><br>

## **Cleanup**

---

Clean up intermediate files

In [ ]:

#fill in 
# os.remove()

In [ ]:
#stop

In [ ]:
# df = df_default_cards
df = df_oracle_cards
eda.copi_df(df)
df.info()